In [26]:
#Importing Necessary Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#Load Data

In [27]:
file_path = '../Data/cierra_updated.csv'
df = pd.read_csv(file_path)

Checking Dimensions and Structure

In [25]:
print("Dataset Dimensions:", df.shape)
print("\nDataset Information:")
df.info()


Dataset Dimensions: (2479, 9)

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category     2479 non-null   object
 1   title        2479 non-null   object
 2   price        2478 non-null   object
 3   link         2479 non-null   object
 4   image        2477 non-null   object
 5   description  2340 non-null   object
 6   top          1920 non-null   object
 7   middle       1530 non-null   object
 8   base         1233 non-null   object
dtypes: object(9)
memory usage: 174.4+ KB


    Column Name Description

 0   category:   specify gender target i.e women, men or unisex

 1   title:      The name of the perfume

 2   price:      The price of the perfume 

 3   link:       The specific location path on Cierra website

 4   image:      The image of the specific perfume

 5   description: A synopsis of the perfume

 6   top:        The initial scent, lasting 5-15 minutes. 

 7   middle:     The core of the fragrance, emerging after the top notes dissipate
 
 8   base:       The final lingering notes, which emerge after the middle notes fade and can last several hours.

Checking for Missing Values

In [28]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
print("Columns with Missing Values:\n", missing_values)

Columns with Missing Values:
 price             1
image             2
description     139
top             559
middle          949
base           1246
dtype: int64


Summary Statistics

In [30]:
print("Summary Statistics:")
df.describe(include='all')

Summary Statistics:


,category,title,price,link,image,description,top,middle,base
count,2479,2479,2478,2479,2477,2340,1920,1530,1233
unique,3,2474,472,2476,2463,2257,1697,1417,1165
top,Women,"PARFUMS DE MARLY, PERSEUS EDP 125ML M","KSh14,500.00",https://cierraperfumes.com/product/xerjoff-kem...,https://i0.wp.com/cierraperfumes.com/wp-conten...,Thanks to the intensely captivating scent of J...,Notes,Notes:,HEART NOTES
freq,1000,2,30,2,2,2,31,19,7
